# Artificial Intelligence

## Tic Tac Toe Heuristics

Having the whole board as a node, and edges connects boards that are possible moves is the best way for an AI to play tic tac toe.

You can easily remove moves that are certain to be useless, for example placing a `o` where there is already an `x`. This is known as **pruning the search tree**.

You also need to look for moves that will prevent the other player to win, for example blocking the third position of a row where the opponent could win on the next move. This is known as **adversarial search**.
The **mini-max algorithm** is used for maximizing your chances of winning on your turn, and your opponent is trying to minimize your chances of winning on their turn.

## What is Intelligence ?

When we cant explain some behavior we say it is intelligent, if we can explain things then it is an algorithm etc..
We consider ourselves intelligent because we dont understand how the brain works, but the day we do, will we be still "intelligent" ?

We can be intelligent at doing a specific tasks, and humans are just good at doing a lot of things.

## Agent, environment, perception, cognition

An agent use his sensors to get **perception** of the environment. It can then take **actions**, and the **cognition** is what makes it decides what actions to take.

An **environment** can be **fully** or **partially observable**. It can also be **deterministic** or **stochastic** (with uncertainty). **Discrete or continuous**. **Benign** (the agent is the only one taking action) or **adversarial** (with other agents).

**Rational Behavior** is taking actions to maximize his desired goal, there is a sense of acting optimally. When behaving optimal is not possible because of constraints, we are in the case of **bounded optimality** where we want to reach boundaries that we fix, like winning 60% of the time.

## Solving a Sudoku with AI

### What is a Sudoku?

Sudoku is one of the world's most popular puzzles. It consists of a 9x9 grid, and the objective is to fill the grid with digits in such a way that each row, each column, and each of the 9 principal 3x3 subsquares contains all of the digits from 1 to 9. The detailed rules can be found, for example, [here](http://www.conceptispuzzles.com/?uri=puzzle/sudoku/rules).

<center><img src="AI/sudoku.png" width=20% /></center>

Solving Sudoku implies to have **constraint propagation** (can narrow the range of possible solutions) and implement a **search** algorithm to explore the different possibilities.

<center><img src="AI/solvedsudoku.png" width=50% /></center>

### Boxes, Units and Peers

And let's start naming the important elements created by these rows and columns that are relevant to solving a Sudoku:

* The individual squares at the intersection of rows and columns will be called boxes. These boxes will have labels 'A1', 'A2', ..., 'I9'.
* The complete rows, columns, and 3x3 squares, will be called units. Thus, each unit is a set of 9 boxes, and there are 27 units in total.
* For a particular box (such as 'A1'), its peers will be all other boxes that belong to a common unit (namely, those that belong to the same row, column, or 3x3 square).



In [78]:
class Sudoku():
    def __init__(self, sudoString):
        self.rawString = sudoString
        
        self.rows = 'ABCDEFGHI'
        self.cols = '123456789'
        self.boxes = self.cross(self.rows, self.cols)
        
        self.rowUnits = [self.cross(r, self.cols) for r in self.rows]
        self.columnUnits = [self.cross(self.rows, c) for c in self.cols]
        self.squareUnits = [self.cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
        
        self.unitlist = self.rowUnits + self.columnUnits + self.squareUnits
        self.units = dict((s, [u for u in self.unitlist if s in u]) for s in self.boxes)
        self.peers = dict((s, set(sum(self.units[s],[]))-set([s])) for s in self.boxes)
        
        self.sudoGrid = self.makeGridDict(sudoString)
    
    def cross(self, rows, cols):
        return [s+t for s in rows for t in cols]
    
    def makeGridDict(self, sudoString):
        assert len(sudoString) == 81, "Input grid must be a string of length 81 (9x9)"
        gridDict = dict(zip(self.boxes, sudoString))

        return gridDict
            
    def solveSudoku(self):
        """
        This algorithm is working for simple Sudoku
        For harder one we will need to implement a search algorithm.
        """
        # insert all the values in the empty box
        solvedGrid = {box:value if value != "." else "123456789" for box, value in self.sudoGrid.items()}
        solvedValues = [box for box in solvedGrid.keys() if len(solvedGrid[box]) == 1]
        
        # eliminate the values that are already placed
        while len(solvedValues) < 81:
            for box in solvedValues:
                for peer in self.peers[box]:
                    solvedGrid[peer] = solvedGrid[peer].replace(solvedGrid[box], "")
            if len(solvedValues) == len([box for box in solvedGrid.keys() if len(solvedGrid[box]) == 1]):
                print("Need to use search algorithm to finish\n")
                
                return False
            solvedValues = [box for box in solvedGrid.keys() if len(solvedGrid[box]) == 1]
        
        self.sudoGrid = solvedGrid
        self.displaySudoGrid()
        return True

    def searchSolve(self):
        """
        The search algorithm tries to pick one solution from the choice that has the 
        fewest numbers to try at random and see if it can solve the Sudoku.
        If not, it rolls back and tries another solution.
        """
        n,s = min((len(self.sudoGrid[s]), s) for s in self.boxes if len(self.sudoGrid[s]) > 1)
        
        for value in self.sudoGrid[s]:
            self.sudoGrid[s] = self.sudoGrid[s].replace(value, "")
            if self.solveSudoku() == False:
                self.searchSolve()
            else:
                print("Solved !")
                return True

    def displaySudoString(self):
        newString = ""
        for i, c in enumerate(self.rawString, start=1):
            newString += c
            newString += " "
            if i % 9 == 0:
                newString += "\n"
            elif i % 3 == 0:
                newString += "|"
            if i % 27 == 0 and i != 81:
                newString += "------+------+------\n"
        return newString
        
    def displaySudoGrid(self):
        width = 1+max(len(self.sudoGrid[s]) for s in self.boxes)
        line = '+'.join(['-'*(width*3)]*3)
        for r in self.rows:
            print(''.join(self.sudoGrid[r+c].center(width)+('|' if c in '36' else '') for c in self.cols))
            if r in 'CF': 
                print(line)

simple = Sudoku('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
hard = Sudoku('4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......')

In [79]:
# print(simple.displaySudoString())
# print(simple.solveSudoku())

print(hard.displaySudoString())
print(hard.solveSudoku())
hard.searchSolve()

4 . . |. . . |8 . 5 
. 3 . |. . . |. . . 
. . . |7 . . |. . . 
------+------+------
. 2 . |. . . |. 6 . 
. . . |. 8 . |4 . . 
. . . |. 1 . |. . . 
------+------+------
. . . |6 . 3 |. 7 . 
5 . . |2 . . |. . . 
1 . 4 |. . . |. . . 

Need to use search algorithm to finish

   4      1679   12679  |  139     2369    1269  |   8      1239     5    
 26789     3    1256789 | 14589   24569  1245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569 1245689 | 12369   12349   123469 
------------------------+------------------------+------------------------
  3789     2     135789 |  3459   34579    4579  | 13579     6     13789  
  3679   15679   135679 |  359      8     25679  |   4     12359   12379  
 36789   456789  356789 |  3459     1     245679 | 23579   23589   23789  
------------------------+------------------------+------------------------
  289      89     289   |   6      459      3    |  1259     7     12489  
   5      6789   36789  |   2      479    14789  |  1